<a href="https://colab.research.google.com/github/JamesLabUofT/GEE_Workshop/blob/main/scripts/working_with_era5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with ERA5 Climate Data in Google Earth Engine Using Python

### Module Overview:

This module introduces participants to the fundamentals of accessing, processing, and extracting climate data—specifically ERA5 reanalysis data—using Google Earth Engine (GEE) and Python. Through hands-on coding exercises in Google Colab, participants will learn how to build scalable workflows for climate data analysis relevant to ecological and environmental research.

### Learning Objectives:

By the end of this module, participants will be able to:

* Understand the structure and content of ERA5 datasets available in GEE.
* Use Python and the Earth Engine API to access and filter ERA5 data by time and region.
* Extract and visualize climate variables (e.g., precipitation, temperature) over custom geographic areas.
* Export processed data for further analysis or reporting.

### Topics Covered:

**Introduction to ERA5 Data**

1. What is ERA5?
2. Key variables and temporal resolution
3. Accessing ERA5 in Google Earth Engine
4. Setting Up the Python Environment

Using Google Colab for cloud-based analysis
Installing and importing required packages (earthengine-api, geemap, pandas, matplotlib)

**Data Acquisition**

1. Filtering ERA5 ImageCollections by date and region
2. Selecting relevant climate variables (e.g., total_precipitation, temperature_2m)

**Data Processing**

1. Reducing data over regions of interest (e.g., national parks, watersheds)
2. Aggregating data temporally (monthly, yearly averages)
3. Converting units and formatting results

**Data Extraction and Visualization**

1. Exporting time series data to Pandas DataFrames**
2. Creating plots and maps using matplotlib and geemap
3. Exporting data to CSV or Google Drive

**Custom Functions to Extract Data**

1. Function to extract parameters for FWI calculations


## Custom Function To Extract FWI Data

In [ ]:
def extract_era5_land_data(feature_collection, year):
    """This function takes a feature collection and a year as YYYY and returns:
       a pandas dataframe with a row for date, t, rh,ws, ppt.
       start date and end date are set in the function to April 30 - August 31 of each year.
       This can be adjusted in the function under #Define the date range

    """
    # Define the date range
    start_date = f'{year}-04-30'
    end_date = f'{year}-08-31'

    # Load the ERA5-Land hourly dataset
    era5_land = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')

    # Filter the dataset by date range and time (13:00)
    era5_filtered = era5_land.filterDate(start_date, end_date).filter(ee.Filter.eq('hour', 13))

    # Define the bands to extract
    bands = ['temperature_2m', 'total_precipitation', 'u_component_of_wind_10m',
             'v_component_of_wind_10m', 'dewpoint_temperature_2m']

    # Select the bands
    era5_selected = era5_filtered.select(bands)

    # Calculate relative humidity and wind speed
    def calculate_additional_bands(image):
        temp = image.select('temperature_2m')
        dewpoint = image.select('dewpoint_temperature_2m')
        u_wind = image.select('u_component_of_wind_10m')
        v_wind = image.select('v_component_of_wind_10m')

        e_temp = temp.expression(
            '6.112 * exp((17.67 * temp) / (temp + 243.5))',
            {'temp': temp}
        )

        e_dew = dewpoint.expression(
            '6.112 * exp((17.67 * dewpoint) / (dewpoint + 243.5))',
            {'dewpoint': dewpoint}
        )

        rh = e_dew.divide(e_temp).multiply(100).rename('relative_humidity_2m')

        wind_speed = u_wind.pow(2).add(v_wind.pow(2)).sqrt().rename('wind_speed_10m')

        return image.addBands([rh, wind_speed])

    # Apply the relative humidity calculation
    era5_with_rh = era5_selected.map(calculate_additional_bands)

    # Function to reduce each image to a dictionary of values
    def reduce_image(image, feature):
        reduced = image.reduceRegion(
            reducer=ee.Reducer.mode(),
            geometry=feature.geometry(),
            scale=1000,
            maxPixels=1e9
        )
        return ee.Feature(None, reduced).set('date', image.date().format('YYYY-MM-dd')).copyProperties(feature, ['Fire_ID', 'defoltd', 'id'])

    # Function to process each feature in the collection
    def process_feature(feature):
        era5_reduced = era5_with_rh.map(lambda image: reduce_image(image, feature))
        return era5_reduced

    # Apply the processing function to each feature in the collection
    processed_collection = feature_collection.map(process_feature).flatten()

    # Convert the collection to a list of dictionaries
    data_list = processed_collection.getInfo()['features']

    # Extract properties from each feature
    data_dicts = [feature['properties'] for feature in data_list]

    # Create a DataFrame
    df = pd.DataFrame(data_dicts)

    df = df[['date', 'temperature_2m', 'relative_humidity_2m', 'wind_speed_10m',
             'total_precipitation', 'Fire_ID', 'defoltd', 'id']]

    def kelvin_to_celcius(x):
        return x - 273.15

    df['temperature_2m'] = df['temperature_2m'].apply(kelvin_to_celcius)

    return df